# Understanding PLONK (Part 1): Plonkish Arithmetization


Arithmetization refers to the process of transforming computations into probabilistically checkable statements for the purpose of zero-knowledge proofs. Plonkish arithmetization is a specific arithmetization method unique to the Plonk proof system. Before the advent of Plonkish, the mainstream circuit representation was R1CS (Rank-1 Constraint Systems), widely adopted by systems like Pinocchio, Groth16, and Bulletproofs. In 2019, the Plonk scheme proposed a seemingly retro circuit encoding method. However, by leveraging polynomial encoding to its fullest, Plonk is no longer limited to just "addition gates" and "multiplication gates" in arithmetic circuits. Instead, it supports more flexible "custom gates" and "lookup gates."

First, let's revisit R1CS circuit encoding, which is the most widely discussed arithmetization scheme. Then, we will introduce Plonkish encoding in comparison.

## Arithmetic Circuits and R1CS Arithmetization

An arithmetic circuit consists of multiple multiplication and addition gates. Each gate has two input wires and one output wire. Any output wire can be connected to multiple input wires of other gates.

Consider a very simple arithmetic circuit:

```
          out
           |
           *
         /   \
       /       \
     /           \
    +             *
  /   \         /   \
 /     \       /     \
x1     x2     x3      2
```

This circuit represents the following computation:

$$
(x_1 + x_2) \cdot (2\cdot x_3) = out
$$

The circuit has four variables: three input variables $(x_1, x_2, x_3)$ and one output variable $out$. Additionally, there is a constant input with a value of $2$.

A circuit has two states: a "blank state" and a "computation state." When the input variables have no specific values, the circuit is in the "blank state." In this state, we can only describe the relationships between the circuit's wires, i.e., the circuit's structural topology.

The next question is how to encode the "blank state" of the circuit, i.e., encode the positions of the gates and the connections between their wires.

R1CS uses the multiplication gates as the central focus, employing three "selector" matrices to "select" which variables are connected to the "left input," "right input," and "output" of each multiplication gate.

Let's first look at the left input of the top multiplication gate in the diagram, which can be described using the following table:

$$
\begin{array}{|c|c|c|c|c|}
\hline
1 & x_1 & x_2 & x_3 & out \\
\hline
0 & 1 & 1 & 0 & 0 \\
\hline
\end{array}
$$

This table has only one row, so we can represent it with a vector $U=(0,1,1,0,0)$ indicating that the left input of the multiplication gate is connected to two variables, $x_1$ and $x_2$. Note that all addition gates are expanded into the sum (or linear combination) of multiple variables.

Next, consider the right input, which is connected to one variable $x_3$ and a constant value, equivalent to connecting $x_3$ multiplied by two. Thus, the right input selector matrix can be written as:

$$
\begin{array}{|c|c|c|c|c|}
\hline
1 & x_1 & x_2 & x_3 & out \\
\hline
0 & 0 & 0 & 2 & 0 \\
\hline
\end{array}
$$

Here, we can also use a row vector $V=(0,0,0,2,0)$ to represent this, where the $2$ corresponds to the constant wire in the circuit diagram.

Finally, the output of the multiplication gate can be described as $W=(0,0,0,0,1)$，producing the output variable $out$：

$$
\begin{array}{|c|c|c|c|c|}
\hline
1 & x_1 & x_2 & x_3 & out \\
\hline
0 & 0 & 0 & 0 & 1 \\
\hline
\end{array}
$$

With these three vectors $(U,V,W)$，we can constrain the circuit's computation using an "inner product" equation:

$$
\big(U\cdot(1,x_1, x_2,x_3,out)\big) \cdot \big(V\cdot(1,x_1, x_2,x_3,out)\big) = \big(W\cdot(1,x_1, x_2,x_3,out)\big)
$$

Simplifying this equation yields:

$$
(x_1 + x2) \cdot (2\cdot x_3) = out
$$

If we substitute the variables with an assignment vector $(1,x_1,x_2,x_3,out) = (1,3,4,5,70)$，the circuit's computation can be verified using the "inner product" equation:

$$
(U\cdot(1,3,4,5,70))\cdot(U\cdot(1,3,4,5,70))=W\cdot(1,3,4,5,70)
$$


In [23]:
import numpy as np

# selector matrices
A = np.array([[0, 1, 1, 0, 0]])
B = np.array([[0, 0, 0, 2, 0]])
C = np.array([[0, 0, 0, 0, 1]])

# assignments vector: (1, x1, x2, x3, out)
assignments = [1, 3, 4, 5, 70]

# calculate inner product, which is element-wise multiplication
inner_product_c = C.dot(assignments)
inner_product_a_b = A.dot(assignments) * B.dot(assignments)

print("Output:")
print(inner_product_c)
print(inner_product_a_b)

assert np.array_equal(inner_product_c, inner_product_a_b)

print("Checks pass!") # Will print this, all checks will pass

Output:
[70]
[70]
Checks pass!


However, an incorrect assignment vector, such as $(1,3,4,\fbox{0},70)$ ，does not satisfy the "inner product equation":

$$
(U\cdot(1,3,4,\fbox{0},70))\cdot(U\cdot(1,3,4,\fbox{0},70))\neq W\cdot(1,3,4,\fbox{0},70)
$$

The result of the left-hand side computation is $0$, while the right-hand side computation yields $70$. Of course, we can verify that $(1,3,4,0,0)$ is also a valid assignment (satisfying the circuit constraints).

In [24]:
import numpy as np

# selector matrices
A = np.array([[0, 1, 1, 0, 0]])
B = np.array([[0, 0, 0, 2, 0]])
C = np.array([[0, 0, 0, 0, 1]])

# assignments vector: (1, x1, x2, x3, out)
assignments = [1, 3, 4, 0, 70]

# calculate inner product, which is element-wise multiplication
inner_product_c = C.dot(assignments)
inner_product_a_b = A.dot(assignments) * B.dot(assignments)

print("Output:")
print(inner_product_c)
print(inner_product_a_b)

assert not np.array_equal(inner_product_c, inner_product_a_b)

print("You have the wrong assignments!")

Output:
[70]
[0]
You have the wrong assignments!


Not every circuit has a valid assignment vector. A circuit that has a valid assignment vector is called a satisfiable circuit. Determining whether a circuit is satisfiable is an NP-Complete problem and also an NP-hard problem.

In this example, the two multiplication gates are not the same. The upper multiplication gate has variables in both its left and right inputs, while the lower multiplication gate has a variable on only one side of its input and a constant on the other. For the latter type of "constant multiplication gate," we will later treat them as special "addition gates," as shown in the figure below. The lower-right multiplication gate in the top circuit is equivalent to the lower-right addition gate in the bottom circuit.

```
          out
           |
           *
         /   \
       /       \
     /           \
    +             *
  /   \         /   \
 /     \       /     \
x1     x2     x3      2
```

```
          out
           |
           *
         /   \
       /       \
     /           \
    +             *
  /   \         /   \
 /     \       /     \
x1     x2     x3 -----\
```

Therefore, if a circuit contains more than one multiplication gate, we cannot represent the computation using the inner product relationship among the three vectors $U, V$, and $W$. Instead, we need to construct a computational relationship involving "three matrices."

### Multiple Multiplication Gates

For example, consider the circuit shown below, which has two multiplication gates, both of which involve variables in their left and right inputs.

```
          out
           |
           * (#1)
         /   \
       /       \   x5
     /           \
    +             * (#2)  
  /   \         /   \
 /     \       /     \
x1     x2     x3      x4
```

This circuit represents the following computation:

$$
(x_1 + x2) \cdot (x3 \cdot x4) = out
$$

We encode the circuit based on the multiplication gates. The first step is to number the multiplication gates in the circuit sequentially (the order of numbering does not matter, as long as it is consistent). The two multiplication gates in the figure are labeled `#1` and `#2`.

Next, we need to assign variable names to the intermediate wires of each multiplication gate. For example, the four input variables are denoted as $x_1, x_2, x_3, x_4$​, where $x_5$​ is the output of the second multiplication gate and also serves as the right input of the first multiplication gate. The output of the first multiplication gate is $out$. Thus, we obtain a vector of variable names:

$$
(x_1, x_2, x_3, x_4, x_5, out)
$$

The "blank state" of this circuit can be encoded using the following three matrices:

$$
U, V, W \in \mathbb{F}^{n\times m}
$$

where $n$ is the number of multiplication gates, and $m$ is roughly the number of wires. Each row of these matrices "selects" the input and output variables of the corresponding multiplication gate. For example, we define the left input matrix $U$ as:

$$
\begin{array}{|c|c|c|c|c|}
\hline
x_1 & x_2 & x_3 & x_4 & x_5 & out & \texttt{i} \\
\hline
1 & 1 & 0 & 0 & 0 & 0 & \texttt{1}\\
\hline
0 & 0 & 1 & 0 & 0 & 0 & \texttt{2}\\
\hline
\end{array}
$$

Here, the left input of the first multiplication gate is $(x_1+x_2)$), and the left input of the second multiplication gate is $x_3$. The right input matrix $V$ is defined as:

$$
\begin{array}{|c|c|c|c|c|}
\hline
x_1 & x_2 & x_3 & x_4 & x_5 & out &\texttt{i}\\
\hline
0 & 0 & 0 & 0 & 1 & 0 & \texttt{1}\\
\hline
0 & 0 & 0 & 1 & 0 & 0 & \texttt{2}\\
\hline
\end{array}
$$

Here, the right input of the first multiplication gate is $x_5$​, and the right input of the second multiplication gate is $x_4$​. Finally, the output matrix $W$ is defined as:

$$
\begin{array}{|c|c|c|c|c|}
\hline
x_1 & x_2 & x_3 & x_4 & x_5 & out & \texttt{i}\\
\hline
0 & 0 & 0 & 0 & 0 & 1 & \texttt{1}\\
\hline
0 & 0 & 0 & 0 & 1 & 0 & \texttt{2}\\
\hline
\end{array}
$$

We treat all wire assignments as a vector $\vec{a}$ (here, the letter $a$ stands for Assignments).

In the above example, the "assignment vector" is:

$$
\vec{a} = (x_1, x_2, x_3,x_4,x_5,out)
$$

Thus, we can easily verify the following equation:

$$
(U \cdot \vec{a}) \circ (V \cdot \vec{a}) = (W \cdot\vec{a})
$$

where the symbol $\circ$ represents the Hadamard Product, which denotes "element-wise multiplication." Expanding the element-wise multiplication equation, we obtain the computational process of the circuit:

$$
\left[
\begin{array}{c}
x_1 + x_2 \\
x_3 \\
\end{array}
\right]
\circ
\left[
\begin{array}{c}
x_5 \\
x_4 \\
\end{array}
\right]=
\left[
\begin{array}{c}
out \\
x_5 \\
\end{array}
\right]
$$

Note that the "assignment vector" typically requires a fixed variable assigned the value $1$ to handle constant inputs in addition gates.

In [25]:
import numpy as np

# selector matrices
A = np.array([[1, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]])
B = np.array([[0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0]])
C = np.array([[0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0]])

# assignments vector: (x1, x2, x3, x4, x5, out)
assignments = [3, 4, 5, 2, 10, 70]

# calculate inner product, which is element-wise multiplication
inner_product_c = C.dot(assignments)
inner_product_a_b = A.dot(assignments) * B.dot(assignments)

print("Output:")

print(inner_product_c)
print(inner_product_a_b)

assert np.array_equal(inner_product_c, inner_product_a_b)

print("Checks pass!") # Will print this, all checks will pass

Output:
[70 10]
[70 10]
Checks pass!


Exercise: Multiple Multiplication Gates

```
          out
           |
           *
         /   \
  x5   /       \   x6
     /           \
    *             *
  /   \         /   \
 /     \       /     \
x1     x2     x3      x4
```

This circuit represents the following computation:

$$
(x_1 \cdot x2) \cdot (x3 \cdot x4) = out
$$

Define the left input matrix $U$ of the circuit：

$$
\begin{array}{c|c|c|c|c|c|c|c}
\hline
x_1 & x_2 & x_3 & x_4 & x_5 & x_6 & out & \texttt{i} \\
\hline
0 & 0 & 0 & 0 & 1 & 0 & 0 & \texttt{1}\\
\hline
1 & 0 & 0 & 0 & 0 & 0 & 0 & \texttt{2}\\
\hline
0 & 0 & 1 & 0 & 0 & 0 & 0 & \texttt{3}\\
\hline
\end{array}
$$

Define the right input matrix $V$ of the circuit：

$$
\begin{array}{c|c|c|c|c|c|c|c}
\hline
x_1 & x_2 & x_3 & x_4 & x_5 & x_6 & out & \texttt{i} \\
\hline
0 & 0 & 0 & 0 & 0 & 1 & 0 & \texttt{1}\\
\hline
0 & 1 & 0 & 0 & 0 & 0 & 0 & \texttt{2}\\
\hline
0 & 0 & 0 & 1 & 0 & 0 & 0 & \texttt{3}\\
\hline
\end{array}
$$


Finally, define the output matrix $W$:

$$
\begin{array}{c|c|c|c|c|c|c|c}
\hline
x_1 & x_2 & x_3 & x_4 & x_5 & x_6 & out & \texttt{i} \\
\hline
0 & 0 & 0 & 0 & 0 & 0 & 1 & \texttt{1}\\
\hline
0 & 0 & 0 & 0 & 1 & 0 & 0 & \texttt{2}\\
\hline
0 & 0 & 0 & 0 & 0 & 1 & 0 & \texttt{3}\\
\hline
\end{array}
$$

In [26]:
import numpy as np

# selector matrices
A = np.array([[0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0],[0, 0, 1, 0, 0, 0, 0]])
#B = np.array([[0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0],[0, 0, 1, 0, 0, 0, 0]])
B = np.array([[0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0],[0, 0, 0, 1, 0, 0, 0]])
C = np.array([[0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0],[0, 0, 0, 0, 0, 1, 0]])

# assignments vector: (x1, x2, x3, x4, x5,x6, out)
assignments = [1, 2, 3, 4, 2, 12, 24]

# calculate inner product, which is element-wise multiplication
inner_product_c = C.dot(assignments)
inner_product_a_b = A.dot(assignments) * B.dot(assignments)

print("Output:")

print(inner_product_c)
print(inner_product_a_b)

assert np.array_equal(inner_product_c, inner_product_a_b)

print("Checks pass!") # Will print this, all checks will pass

Output:
[24  2 12]
[24  2 12]
Checks pass!


### Pros and Cons

Since R1CS encoding is centered around multiplication gates, the addition gates in the circuit do not increase the number of rows in the $U, V$, and $W$ matrices, thus having little impact on the Prover's performance. The encoding of R1CS circuits is clear and simple, making it conducive to constructing various SNARK schemes on top of it.

In the 2019 Plonk paper, the encoding scheme requires encoding both addition and multiplication gates, which seemingly increases the number of constraints and reduces proving performance. However, the Plonk team subsequently introduced gates beyond multiplication and addition, such as gates for range checks, XOR operations, and more. Moreover, Plonk supports any gate whose input-output relationship satisfies a polynomial equation, known as Custom Gates, as well as state transition gates suitable for implementing RAM. With the introduction of lookup gates, Plonk has gradually become the preferred scheme for many applications, and its encoding method has been given a specific name: Plonkish.

## Plonkish Arithmetic Gates

Let's revisit the example circuit. We number the three gates as $\texttt{1},\texttt{2},\texttt{3}$，and also label the output of the addition gate as a variable $x_6$.

```
          out
           |
           * #3
         /   \
  x6   /       \   x5
     /           \
    +  #2         *  #1
  /   \         /   \
 /     \       /     \
x1     x2     x3      x4
```

The circuit encodes the following three constraints:

- $x_1 + x_2 =x_6$
- $x_3\cdot x_4 = x_5$
- $x_6 \cdot x_5 = out$

We define a matrix $W\in\mathbb{F}^{n\times 3}$ to represent the constraints（where $n$ is the number of arithmetic gates）：

$$
\begin{array}{c|c|c|c|}
\texttt{i} & w_a & w_b & w_c  \\
\hline
\texttt{1} & x_6 & x_5 & out \\
\texttt{2} & x_1 & x_2 & x_6 \\
\texttt{3} & x_3 & x_4 & x_5 \\
\end{array}
$$

To distinguish between addition and multiplication, we define a vector $Q\in\mathbb{F}^{n\times5}$ to represent the operators:

$$
\begin{array}{c|c|c|c|}
\texttt{i}  & q_L & q_R & q_M & q_C & q_O  \\
\hline
\texttt{1} & 0 & 0 & 1 & 0& 1 \\
\texttt{2} & 1 & 1 & 0 & 0& 1 \\
\texttt{3} & 0 & 0 & 1 & 0& 1 \\
\end{array}
$$

We can then represent the three constraints using the following equation:

$$
q_L \circ w_a + q_R \circ w_b + q_M\circ(w_a\cdot w_b) + q_C -  q_O\circ w_c = 0
$$

Substituting and expanding the above equation, we obtain the following constraint equations:

$$
\left[
\begin{array}{c}
0\\
1 \\
0\\
\end{array}
\right]
\circ
\left[
\begin{array}{c}
x_6 \\
x_1 \\
x_5\\
\end{array}
\right]
+
\left[
\begin{array}{c}
0\\
1 \\
0\\
\end{array}
\right]
\circ
\left[
\begin{array}{c}
x_5 \\
x_2 \\
x_4\\
\end{array}
\right]
+
\left[
\begin{array}{c}
1\\
0 \\
1\\
\end{array}
\right]
\circ
\left[
\begin{array}{c}
x_6\cdot x_5 \\
x_1\cdot x_2 \\
x_3\cdot x_4\\
\end{array}
\right]=\left[
\begin{array}{c}
1\\
1 \\
1\\
\end{array}
\right]
\circ
\left[
\begin{array}{c}
out \\
x_6 \\
x_5\\
\end{array}
\right]
$$

Simplifying, we get:

$$
\left[
\begin{array}{c}
0 \\
x_1 \\
0\\
\end{array}
\right]
+
\left[
\begin{array}{c}
0 \\
x_2 \\
0\\
\end{array}
\right]
+
\left[
\begin{array}{c}
x_6\cdot x_5 \\
0 \\
x_3\cdot x_4\\
\end{array}
\right]=\left[
\begin{array}{c}
out \\
x_6 \\
x_5\\
\end{array}
\right]
$$

This precisely corresponds to the computational constraints of the three arithmetic gates.

Fill the matrix $W\in\mathbb{F}^{n\times 3}$ with the corresponding values to satisfy the constraints of each gate in the circuit:

$$
\begin{array}{c|c|c|c|}
\texttt{i} & w_a & w_b & w_c  \\
\hline
\texttt{1} & 3 & 33 & 99 \\
\texttt{2} & 1 & 2 & 3 \\
\texttt{3} & 3 & 11 & 33 \\
\end{array}
$$

Then, verify it in code:

In [27]:
import numpy as np

# Q
q_L = np.array([0, 1, 0])
q_R = np.array([0, 1, 0])
q_M = np.array([1, 0, 1])
q_C = np.array([0, 0, 0])
q_O = np.array([1, 1, 1])

w_a = np.array([3, 1, 3])
w_b = np.array([33, 2, 11])
w_c = np.array([99, 3, 33])

constraints = q_L.dot(w_a) + q_R.dot(w_b) + q_M.dot(w_a * w_b) + q_C.dot(np.array([1, 1, 1])) - q_O.dot(w_c)

assert constraints == 0

Let's try a practice exercise.


```
                  out
                  |
                  *
                /   \
          x6  /       \   x7
            /           \
           *             *
    x2   /   \         /   \
        /     \       /     \
       +      x3     x4     x5
      / \
     /   \
    x1    x1
```

This circuit encodes the following four constraints:

- $x_1 + x_1 =x_2$
- $x_2\cdot x_3 = x_6$
- $x_4 \cdot x_5 = x_7$
- $x_6 \cdot x_7 = out$

First, write down the constraint matrix

$$
\begin{array}{c|c|c|c|}
\texttt{i} & w_l & w_r & w_0  \\
\hline
\texttt{1} & x_1 & x_1 & x_2 \\
\texttt{2} & x_2 & x_3 & x_6 \\
\texttt{3} & x_4 & x_5 & x_7 \\
\texttt{4} & x_6 & x_7 & out \\
\end{array}
$$

Next, define the selector vector $Q\in\mathbb{F}^{n\times5}$ to distinguish between multiplication and addition and represent the operators

$$
\begin{array}{c|c|c|c|}
\texttt{i}  & q_L & q_R & q_M & q_C & q_O  \\
\hline
\texttt{1} & 1 & 1 & 0 & 0& 1 \\
\texttt{2} & 0 & 0 & 1 & 0& 1 \\
\texttt{3} & 0 & 0 & 1 & 0& 1 \\
\texttt{4} & 0 & 0 & 1 & 0& 1 \\
\end{array}
$$

This allows us to represent the four constraints in a single equation:

$$
q_L \circ w_a + q_R \circ w_b + q_M\circ(w_a\cdot w_b) + q_C -  q_O\circ w_c = 0
$$


Substituting and expanding the above equation, we obtain the following constraint equations:

$$
\left[
\begin{array}{c}
1\\
0 \\
0\\
0\\
\end{array}
\right]
\circ
\left[
\begin{array}{c}
x_1 \\
x_2 \\
x_4\\
x_6\\
\end{array}
\right]
+
\left[
\begin{array}{c}
1\\
0 \\
0\\
0\\
\end{array}
\right]
\circ
\left[
\begin{array}{c}
x_1 \\
x_3 \\
x_5\\
x_7\\
\end{array}
\right]
+
\left[
\begin{array}{c}
0\\
1 \\
1\\
1\\
\end{array}
\right]
\circ
\left[
\begin{array}{c}
x_1\cdot x_1 \\
x_2\cdot x_3 \\
x_4\cdot x_5\\
x_6\cdot x_7\\
\end{array}
\right]=\left[
\begin{array}{c}
1\\
1 \\
1\\
1\\
\end{array}
\right]
\circ
\left[
\begin{array}{c}
x_2 \\
x_6 \\
x_7\\
out\\
\end{array}
\right]
$$

After simplifying, we get：

$$
\left[
\begin{array}{c}
x_1\\
0 \\
0\\
0\\
\end{array}
\right]
+
\left[
\begin{array}{c}
x_1 \\
0 \\
0 \\
0\\
\end{array}
\right]
+
\left[
\begin{array}{c}
0 \\
x_2\cdot x_3 \\

x_4\cdot x_5\\
x_6\cdot x_7\\
\end{array}
\right]=\left[
\begin{array}{c}
x_2 \\
x_6 \\
x_7\\
out \\
\end{array}
\right]
$$

This precisely corresponds to the computational constraints of the four arithmetic gates.


Fill the matrix $W\in\mathbb{F}^{n\times 4}$ with the corresponding values, to satisfy each gate's constraint in the circuit:

$$
\begin{array}{c|c|c|c|}
\texttt{i} & w_l & w_r & w_o  \\
\hline
\texttt{1} & 1 & 1 & 2 \\
\texttt{2} & 2 & 3 & 6 \\
\texttt{3} & 4 & 5 & 20 \\
\texttt{4} & 6 & 20 & 120 \\
\end{array}
$$

Then, verify it in code:

In [28]:
import numpy as np

# Q
q_L = np.array([1, 0, 0, 0])
q_R = np.array([1, 0, 0, 0])
q_M = np.array([0, 1, 1, 1])
q_C = np.array([0, 0, 0, 0])
q_O = np.array([1, 1, 1, 1])

w_l = np.array([1, 2, 4, 6])
w_r = np.array([1, 3, 5, 20])
w_o = np.array([2, 6, 20,120])

constraints = q_L.dot(w_l) + q_R.dot(w_r) + q_M.dot(w_l * w_r) + q_C.dot(np.array([1, 1, 1, 1])) - q_O.dot(w_o)

assert constraints == 0

To summarize, Plonkish requires a selector matrix $Q$ to describe the "blank state" of the circuit, while all assignments are written into the $W$ matrix. For the interaction protocol between the Prover and the Verifier, $W$ serves as the Prover's witness, representing secret knowledge that is kept hidden from the Verifier. The $Q$ matrix, on the other hand, represents a mutually agreed-upon description of the circuit.

However, the $Q$ matrix alone is not sufficient to precisely describe the example circuit above.

## Copy Constraints

Consider the following two circuits. Their $Q$ matrices are identical, yet the circuits themselves are entirely different.


```
                  out
                   |
                   * #1
                 /   \
          x6   /       \   x5
             /           \
            +  #2         *  #3
          /   \         /   \
         /     \       /     \
        x1     x2     x3      x4
```

```
                   out
                    |
                    * #1
                  /   \
           x7   /       \   x8
              /           \

    x6                            x5
    |                             |
    |                             |
    +  #2                         *  #3
  /   \                         /   \
 /     \                       /     \
x1     x2                     x3      x4
```

The difference between the two circuits lies in whether $x_5$​ and $x_6$​ are connected to gate `#1`. If the Prover directly fills the circuit assignments into the $W$ table, an "honest" Prover would fill the same value in both $w_{a,1}$​ and $w_{c,2}$​. However, a "malicious" Prover could fill in different values. If the malicious Prover also fills different values in $w_{b,1}$​ and $w_{c,3}$​, then the Prover is effectively proving the circuit on the right side of the figure, rather than the circuit that was agreed upon with the Verifier (the left side).

$$
\begin{array}{c|c|c|c|}
i & w_a & w_b & w_c  \\
\hline
1 & \boxed{x_6} & \underline{x_5} & out \\
2 & x_1 & x_2 & \boxed{x_6} \\
3 & x_3 & x_4 & \underline{x_5} \\
\end{array}
$$

Let's look at an example：

In [29]:
import numpy as np

# Q
q_L = np.array([0, 1, 0])
q_R = np.array([0, 1, 0])
q_M = np.array([1, 0, 1])
q_C = np.array([0, 0, 0])
q_O = np.array([1, 1, 1])

w_a = np.array([3, 1, 3])
w_b = np.array([22, 2, 11]) #replace 33 with 22 
w_c = np.array([66, 3, 33]) #replace 99 with 66; only satisfies the constraints 3*22=66; 1+2=3;and 3*11=33 

constraints = q_L.dot(w_a) + q_R.dot(w_b) + q_M.dot(w_a * w_b) + q_C.dot(np.array([1, 1, 1])) - q_O.dot(w_c)

assert constraints == 0

In [30]:
# Even if an output constraint is added, meaning the output must be 99,
# the prover can still easily construct an appropriate W matrix to pass
# the verification.
import numpy as np

# Q
q_L = np.array([0, 1, 0])
q_R = np.array([0, 1, 0])
q_M = np.array([1, 0, 1])
q_C = np.array([0, 0, 0])
q_O = np.array([1, 1, 1])

w_a = np.array([9, 1, 3])
w_b = np.array([11, 2, 11])  
w_c = np.array([99, 3, 33]) 

constraints = q_L.dot(w_a) + q_R.dot(w_b) + q_M.dot(w_a * w_b) + q_C.dot(np.array([1, 1, 1])) - q_O.dot(w_c)

assert constraints == 0

herefore, we need to introduce new constraints to enforce that in the right circuit diagram, $x_6=x_7$​ and $x_5=x_8$​. This is equivalent to requiring that when the Prover fills the same variable into multiple positions in the table, the values must be equal.

This necessitates a new type of constraint: the Copy Constraint. Plonk uses a permutation argument to ensure that the values in multiple positions of the $W$ matrix satisfy the copy relationship. Let’s continue with the example of the circuit diagram above to illustrate the basic idea:

First, let's rearrange all the cells in the $W$ table into a vector:

$$
\sigma_0=(\boxed{w_{a,1}}, w_{a,2}, w_{a,3}, \underline{w_{b,1}}, w_{b,2}, w_{b,3}, w_{c,1}, \boxed{w_{c,2}}, \underline{w_{c,3}})
$$

Then, we swap the cells that should be equal. For example, in the diagram above, we require $w_{a,1}=w_{c,2}$ and $w_{b,1}=w_{c,3}$. This gives us the following modified position vector:

$$
\sigma=(\boxed{w_{c,2}}, w_{a,2}, w_{a,3}, \underline{w_{c,3}}, w_{b,2}, w_{b,3}, w_{c,1}, \boxed{w_{a,1}}, \underline{w_{b,1}})
$$

We then require the Prover to prove that after permuting the $W$ table as above, it remains equal to itself. The equality before and after the permutation ensures that the Prover cannot cheat.

Here’s another example: when constraining that three (or more) positions in a vector must have the same value, we simply perform a cyclic shift (left or right) of these positions and prove that the shifted vector equals the original vector. For instance:

$$
A = (b_1, b_2, \underline{a_1}, b_3, \underline{a_2}, b_4, \underline{a_3})
$$

If we want to prove $a_1=a_2=a_3$，we need only prove:

$$
A' =  (b_1, b_2, \underline{a_3}, b_3, \underline{a_1}, b_2, \underline{a_2}) \overset{?}{=} A
$$

In the permuted vector $A'$, $a_1, a_2, a_3$ are cyclically shifted to the right, meaning that $a_1$ is placed in the position of $a_2$, $a_2$ in the position of $a_3$, and $a_3$ in the position of $a_1$.

If $A'=A$ ，then all the corresponding positions in $A'$ and $A$ must be equal, which implies： $a_1=a_3$， $a_2=a_1$， $a_3=a_2$，i.e. $a_1=a_2=a_3$. This method can be applied to any number of equivalence relationships. (The method for proving the equality of two vectors will be discussed in the next chapter.)

So, how do we describe the permutation in the circuit assignment table? We only need to record the $\sigma$ vector, which can also be written in table form:

$$
\begin{array}{c|c|c|c|}
i & \sigma_a & \sigma_b & \sigma_c  \\
\hline
1 & \boxed{w_{c,2}} & \underline{w_{c,3}}& w_{c,1} \\
2 & w_{a,2} & w_{b,2} & \boxed{w_{a,1}} \\
3 & w_{a,3} & w_{b,3} & \underline{w_{b,1}} \\
\end{array}
$$

With $\sigma$, the blank circuit can be described as $(Q,\sigma)$, and the circuit assignment as $W$

$$
\mathsf{Plonkish}_0 \triangleq (Q, \sigma; W)
$$

## Further Comparison

In R1CS, the width of the $(U,V,W)$ tables is related to the number of wires, and the number of rows is related to the number of multiplication gates. This construction treats the arithmetic circuit as consisting solely of multiplication gates, but each gate can have multiple input pins (up to the total number of wires). In contrast, Plonkish treats addition and multiplication gates equally, and since each gate has only two input pins, the width of the $W$ table is fixed at three columns (it can be extended to more columns if advanced computation gates are supported). This characteristic is a prerequisite for Plonk to implement copy constraints using the Permutation Argument.

> ..., and thus our linear contraints are just wiring constraints that can be reduced to a permutation check. 

According to the statistics in the Plonk paper, in general, the number of addition gates in an arithmetic circuit is twice the number of multiplication gates. If this is the case, the length of the $W$ table will be three times that of the R1CS matrix. However, this trade-off brings greater flexibility in arithmetization.

## Outline of PlonK IOP

Given our understanding of the blank circuit structure and its assignments, we can outline the rest of the Plonk protocol.

First, the Prover and Verifier agree on a common circuit $(Q,\sigma)$. For the purposes of illustration, assumed the public output of the circuit is $out=99$，and $(x_1,x_2,x_3,x_4)$ are the secret inputs

The Prover fills out the $W$ matrix (invisible to the Verifier):

$$
\begin{array}{c|c|c|c|}
i & w_a & w_b & w_c  \\
\hline
1 & \boxed{x_6} & \underline{x_5} & [out] \\
2 & x_1 & x_2 & \boxed{x_6} \\
3 & x_3 & x_4 & \underline{x_5} \\
4 & 0 & 0 & [out] \\
\end{array}
$$

Here, the fourth row is added to introduce an additional arithmetic constraint: $out=99$, explicitly representing the $out$ value in the $Q$ matrix.

The corresponding $Q$ matrix agreed upon by the Prover and Verifier is:

$$
\begin{array}{c|c|c|c|}
i & q_L & q_R & q_M & q_C & q_O  \\
\hline
1 & 0 & 0 & 1 & 0& 1 \\
2 & 1 & 1 & 0 & 0& 1 \\
3 & 0 & 0 & 1 & 0& 1 \\
4 & 0 & 0 & 0 & 99& 1 \\
\end{array}
$$

The fourth constraint enforces $out=99$. Substituting $(q_L=0, q_R=0,q_M=0,q_C=99,q_O=1)$ into the arithmetic constraint yields $99-w_c = 0$, i.e. $w_{c,4}=99$ 。

$$
q_L \circ w_a + q_R \circ w_b + q_M\circ(w_a\cdot w_b) + q_C -  q_O\circ w_c = 0
$$

To enforce $w_c$ in the first row to also be $99$，an additional copy constraint must be added to the $\sigma$ matrix: swap the $out$ cell $(w_{c,1})$ with the $w_{c,4}$ cell:

$$
\begin{array}{c|c|c|c|}
i & \sigma_a & \sigma_b & \sigma_c  \\
\hline
1 & \boxed{w_{c,2}} & \underline{w_{c,3}} & [w_{c,4}] \\
2 & w_{a,2} & w_{b,2} & \boxed{w_{a,1}} \\
3 & w_{a,3} & w_{b,3} & \underline{w_{b,1}} \\
4 & w_{a,4} & w_{b,4} & [w_{c,1}]\\
\end{array}
$$

If the Prover is honest, then for $i\in(1,2,3,4)$，the following arithmetic constraint holds:

$$
q_{L,i} \circ w_{a,i} + q_{R,i} \circ w_{b,i} + q_{M,i}\circ(w_{a,i}\cdot w_{b,i}) + q_{C,i} -  q_{O,i}\circ w_{c,i} = 0
$$


Let the matrix $W$ be:
$$
\begin{array}{c|c|c|c|}
i & w_a & w_b & w_c  \\
\hline
1 & \boxed{3} & \underline{33} & [99] \\
2 & 1 & 2 & \boxed{3} \\
3 & 3 & 11 & \underline{33} \\
4 & 0 & 0 & [99] \\
\end{array}
$$

The implementation is as follows:

In [31]:
import numpy as np

# Q
q_L = np.array([0, 1, 0, 0])
q_R = np.array([0, 1, 0, 0])
q_M = np.array([1, 0, 1, 0])
q_C = np.array([0, 0, 0, 99])
q_O = np.array([1, 1, 1, 1])

w_a = np.array([3, 1, 3, 0])
w_b = np.array([33, 2, 11, 0])
w_c = np.array([99, 3, 33, 99])

constraints = q_L.dot(w_a) + q_R.dot(w_b) + q_M.dot(w_a * w_b) + q_C.dot(np.array([1, 1, 1, 1])) - q_O.dot(w_c)
print("Output:")
print ("constraints: ", constraints)
assert constraints == 0

# or

for i in range(0,4):
    constraint = q_L[i] * w_a[i] + q_R[i] * w_b[i] + q_M[i] * w_a[i] * w_b[i] + q_C[i] - q_O[i] * w_c[i]
    print("constraint", i, ":", constraint)
    assert constraint == 0

Output:
constraints:  0
constraint 0 : 0
constraint 1 : 0
constraint 2 : 0
constraint 3 : 0


The general idea of the verification protocol is as follows:

**Witness generation and commitment**: The Prover truthfully fills out the $W$ table, encodes each column of the $W$ table, performs polynomial commitments, and sends the committed results to the Verifier.

**IOP**: The Verifier and Prover interact further to verify whether the following equation holds:

$$
q_{L}(X) \cdot w_{a}(X) + q_{R}(X) \cdot w_{b}(X) + q_{M}(X)\cdot(w_{a}(X)\cdot w_{b}(X)) + q_{C}(X) -  q_{O}(X)\cdot w_{c}(X) \overset{?}{=} 0
$$

The code below demonstrates how to verify the validity of the above equation. This process involves knowledge of polynomials (Lagrange interpolation) and finite fields. For an explanation of polynomial concepts, refer to the next section.

In [32]:
import numpy as np
import galois

GF101 = galois.GF(101)

# Given two 1-D arrays x and y
# Returns the Lagrange interpolating polynomial through the points (x, y).
def lagrange_poly_in_finite_field(x, y):
    return galois.lagrange_poly(GF101(x), GF101(y))

# Q
# q_L = [0, 1, 0, 0]
q_L = [0, 0, 0, 1]
q_R = [0, 1, 0, 0]
q_M = [1, 0, 1, 0]
q_C = [0, 0, 0, 99]
q_O = [1, 1, 1, 1]

w_a =[3, 1, 3, 0]
w_b =[33, 3, 11, 0]
w_c =[99, 3, 33, 99]

x_points = [0, 1, 2, 3]

# do the interpolation, return polynomial
q_L_x = lagrange_poly_in_finite_field(x_points, q_L)
q_R_x = lagrange_poly_in_finite_field(x_points, q_R)
q_M_x = lagrange_poly_in_finite_field(x_points, q_M)
q_C_x = lagrange_poly_in_finite_field(x_points, q_C)
q_O_x = lagrange_poly_in_finite_field(x_points, q_O)
w_a_x = lagrange_poly_in_finite_field(x_points, w_a)
w_b_x = lagrange_poly_in_finite_field(x_points, w_b)
w_c_x = lagrange_poly_in_finite_field(x_points, w_c)

print("Output:")
print("q_L_x: ", q_L_x)
print("w_a_x:",w_a_x)

print("q_L_x evaluations: ", q_L_x(x_points))
print("q_R_x evaluations: ", q_R_x(x_points))
print("q_M_x evaluations: ", q_M_x(x_points))
print("q_C_x evaluations: ", q_C_x(x_points))
print("q_O_x evaluations: ", q_O_x(x_points))
print("w_a_x evaluations: ", w_a_x(x_points))
print("w_b_x evaluations: ", w_b_x(x_points))
print("w_c_x evaluations: ", w_c_x(x_points))

# Check the equality with original values
assert np.array_equal(q_L_x(x_points), q_L)
assert np.array_equal(q_R_x(x_points), q_R)
assert np.array_equal(q_M_x(x_points), q_M)
assert np.array_equal(q_C_x(x_points), q_C)
assert np.array_equal(q_O_x(x_points), q_O)
assert np.array_equal(w_a_x(x_points), w_a)
assert np.array_equal(w_b_x(x_points), w_b)
assert np.array_equal(w_c_x(x_points), w_c)

final_check_polynomial = (
    q_L_x * w_a_x              # 0
    + q_R_x * w_b_x            # 3
    + q_M_x * (w_a_x * w_b_x)  # 99 + 33
    + q_C_x                    # 99
    - q_O_x * w_c_x            # 99 + 3 + 33 + 99
)

evaluations = final_check_polynomial(x_points)

print("evaluations: ", evaluations)

assert np.array_equal(evaluations, [0, 0, 0, 0])

print("All checks pass!")

Output:
q_L_x:  17x^3 + 50x^2 + 34x
w_a_x: 49x^3 + 57x^2 + 94x + 3
q_L_x evaluations:  [0 0 0 1]
q_R_x evaluations:  [0 1 0 0]
q_M_x evaluations:  [1 0 1 0]
q_C_x evaluations:  [ 0  0  0 99]
q_O_x evaluations:  [1 1 1 1]
w_a_x evaluations:  [3 1 3 0]
w_b_x evaluations:  [33  3 11  0]
w_c_x evaluations:  [99  3 33 99]
evaluations:  [0 0 0 0]
All checks pass!


To develop an intuition for Lagrange interpolation over the Galois field, let's manually interpolate the values of `q_L` and `w_a` at the evaluation points `x_points = [0, 1, 2, 3]`.

First, recall the Lagrange basis polynomials (defined s.t. $L_i(i) = 1, L_j(i) = 0, i \neq j$):
\begin{align*}
L_0(X) &= \frac {(X-1)(X-2)(X-3)} {((0-1)(0-2)(0-3))} \\
       &= 84 \cdot [x^3-6 \cdot x^2 + 11x -6] = 84x^3 +x^2 + 15x +1 \\
L_1(X) &= \frac {X(X-2)(X-3)} {(1-0)(1-2)(1-3)} \\
       &= 51 \cdot [x^3 - 5 \cdot x^2 + 6  \cdot x] = 51x^3 +48x^2 + 3x \\
L_2(X) &= \frac {X(X-1)(X-3)} {((2-0)(2-1)(2-3))} \\
       &= 50 \cdot [x^3 -4 \cdot x^2 + 3  \cdot x] = 50x^3 +2x^2 + 49x \\
L_3(X) &= \frac {X(X-1)(X-2)} {((3-0)(3-1)(3-2))} \\
       &= 17 \cdot [x^3 - 3 \cdot x^2 + 2  \cdot x] = 17x^3 + 50x^2 + 34x
\end{align*}

For `q_L = [0, 1, 0, 0]`, the interpolation polynomial $q_L(X)$ must satisfy: $$q_L(0)=0; q_L(1)=1; q_L(2)=0; q_L(3)=0.$$ We can construct it as a simple linear combination of the evaluation points and the Lagrange basis polynomials:

\begin{align*}
P(x) &= 0 \cdot L_0(X) + 1 \cdot L_1(X)  + 0 \cdot L_2(X) + 0 \cdot L_3(X) \\
     &= 51x^3 +48x^2 + 3x
\end{align*}

Similarly, for `w_a = [3 1 3 0]`, we need $w_a(X)$ s.t.:
$w_a(0)=3; w_a(1)=1; w_a(2)=3; w_a(3)=0.$$

This can again be constructed as a linear combination of Lagrange basis polynomials:

\begin{align*}
P(x) &= 3 \cdot L_0(X) + 1 \cdot L_1(X)  + 3 \cdot L_2(X) + 0 \cdot L_3(X) \\
     &= 3 \cdot (84x^3 +x^2 + 15x +1) + (51x^3 +48x^2 + 3x)+ 3 \cdot (50x^3 +2x^2 + 49x) \\
     &= 49x^3+57x^2+94x+3
\end{align*}


Of course, this verification is not yet complete. We also need to verify the relationship between that $(w_a(X),w_b(X),w_c(X))$ satisfy the copy constraints $(\sigma_a(X),\sigma_b(X),\sigma_c(X))$. Further, the IOP and the use of polynomial commitments will be discussed in the subsequent chapters.

## Reference materials

- [BG12] Bayer, Stephanie, and Jens Groth. "Efficient zero-knowledge argument for correctness of a shuffle." *Annual International Conference on the Theory and Applications of Cryptographic Techniques*. Springer, Berlin, Heidelberg, 2012.
- [GWC19] Ariel Gabizon, Zachary J. Williamson, and Oana Ciobotaru. "Plonk: Permutations over lagrange-bases for oecumenical noninteractive arguments of knowledge." *Cryptology ePrint Archive* (2019).


----